In [25]:
import requests
import json
import pandas as pd
from opnieuw import retry
from requests.exceptions import HTTPError, ConnectionError
import time
from tinydb import TinyDB, Query

#### Functions

In [3]:
# Get all INE available operations
@retry(
    retry_on_exceptions=(ConnectionError, HTTPError),
    max_calls_total=4,
    retry_window_after_first_call_in_seconds=60,
)
def get_operaciones_disponibles():
    url = "https://servicios.ine.es/wstempus/js/ES/OPERACIONES_DISPONIBLES"
    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code == 200:
        return response.json()


In [4]:
# Get data as series
@retry(
    retry_on_exceptions=(ConnectionError, HTTPError),
    max_calls_total=4,
    retry_window_after_first_call_in_seconds=60,
)
def get_datos(id_operacion, offset=1):
    url = f"https://servicios.ine.es/wstempus/js/ES/SERIES_OPERACION/{id_operacion}?page={offset}"
    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code == 200:
        return response.json()

In [5]:
# Get data from a series
@retry(
    retry_on_exceptions=(ConnectionError, HTTPError),
    max_calls_total=4,
    retry_window_after_first_call_in_seconds=60,
)
def get_series(cod, nult=100):
    url = f"https://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{cod}?nult={nult}"
    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code == 200:
        return response.json()


In [6]:
def iterate_operacion(id_operacion):
    responses = []
    offset = 1
    response = get_datos(id_operacion, offset)
    responses += response
    while True:
        offset += 1
        print(f'Iteracion en pagina {offset}')
        response = get_datos(id_operacion, offset)
        responses += response
        if len(response)<500:
            break
    return responses

#### Main

In [7]:
res = get_operaciones_disponibles()

In [8]:
df_operaciones = pd.DataFrame(res)

In [9]:
df_operaciones.to_csv('operaciones_disponibles.csv', index=False)

In [10]:
df_operaciones[df_operaciones['Nombre'] == 'Contabilidad Nacional de España. Base 2010']

,Id,Cod_IOE,Nombre,Codigo,Url
66,247,30023,Contabilidad Nacional de España. Base 2010,CNE,NaN


In [12]:
df_series = pd.DataFrame(iterate_operacion(247))

Iteracion en pagina 2
Iteracion en pagina 3
Iteracion en pagina 4
Iteracion en pagina 5


In [13]:
df_series.head()

,Id,COD,FK_Operacion,Nombre,Decimales,FK_Periodicidad,FK_Publicacion,FK_Clasificacion,FK_Escala,FK_Unidad
0,1176,CNE1176,247,Industria del papel. Total Nacional. Valor aña...,1,12,460,71,1,135
1,1177,CNE1177,247,"Industria de la madera y del corcho, excepto m...",4,12,460,71,1,135
2,1178,CNE1178,247,"Industria de la madera y del corcho, excepto m...",1,12,460,71,1,135
3,1179,CNE1179,247,"Industria textil, confección de prendas de ves...",4,12,460,71,1,135
4,1180,CNE1180,247,"Industria textil, confección de prendas de ves...",1,12,460,71,1,135


In [14]:
# Check para ver que no hay series vacias
for index, row in df_series.iterrows():
    print(f"Obteniendo datos de {row['COD']}")
    response = get_series(row['COD'])
    if response:
        data = response.get('Data', [])
        if data:
            print(f"{row['COD']} - {response}")
            break

Obteniendo datos de CNE1176
Obteniendo datos de CNE1177


KeyboardInterrupt: 

In [15]:
pd.DataFrame(response['Data'])

""


In [16]:
# Check para ver que no hay series vacias
for index, row in df_series.iterrows():
    print(f"Obteniendo datos de {row['COD']}")
    response = get_series(row['COD'])
    if response:
        data = response.get('Data', [])
        if data:
            print(f"{row['COD']} - {response}")
            break

Obteniendo datos de CNE1176


KeyboardInterrupt: 

In [22]:
df_series.to_excel(r"C:\Users\anasa\Desktop\Ana\IRONHACK\Proyecto final/df_series.xlsx")

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\anasa\\Desktop\\Ana\\IRONHACK\\Proyecto final/df_series.xlsx'

In [27]:
codigo="CNE2444"
db=TinyDB(r"C:\Users\anasa\Desktop\Ana\IRONHACK\Proyecto final/series.json")
series_table=db.table("series")
Serie=Query()
for i in range(0, 5):
    try:
        print(f'Obteniendo datos de {codigo}. iteracion {i+1}')
        response=get_series(codigo)
        if response:
            if response.get('Data', []):
                series_table.upsert(response, Serie.codigo==response["COD"])
                break
    except:
        continue

Obteniendo datos de CNE2444. iteracion 1
Obteniendo datos de CNE2444. iteracion 2
Obteniendo datos de CNE2444. iteracion 3
Obteniendo datos de CNE2444. iteracion 4
Obteniendo datos de CNE2444. iteracion 5


In [21]:
var_PIB=get_series("CNE2444")
var_PIB

{'COD': 'CNE2444',
 'Nombre': 'Total CNAE. Total Nacional. Valor añadido bruto. Precios corrientes. Variación anual. ',
 'FK_Unidad': 7,
 'FK_Escala': 13,
 'Data': []}

In [19]:
pd.DataFrame(var_PIB)

,COD,Nombre,FK_Unidad,FK_Escala,Data


In [ ]:
# Get data as series
@retry(
    retry_on_exceptions=(ConnectionError, HTTPError, ProxyError),
    max_calls_total=4,
    retry_window_after_first_call_in_seconds=60,
)
def get_table(id_tabla, nult=1):
    url = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{id_tabla}?nult={nult}"
    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code == 200:
        return response.json()

NameError: name 'ProxyError' is not defined